In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df= pd.read_csv('OnlineRetail.csv',encoding= 'unicode_escape')

C:\Users\USER\Anaconda3\envs\tf2-py37\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Buying_time,UnitPrice,CustomerID,Country
0,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011/12/9,09:27,2.08,16446.0,United Kingdom
1,556690,23005,printing smudges/thrown away,-9600,2011/6/14,10:37,0.00,NaN,United Kingdom
2,556691,23005,printing smudges/thrown away,-9600,2011/6/14,10:37,0.00,NaN,United Kingdom
3,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010/12/2,14:23,0.03,15838.0,United Kingdom
4,556687,23003,Printing smudges/thrown away,-9058,2011/6/14,10:36,0.00,NaN,United Kingdom


In [4]:
df=df.sort_values(by=['InvoiceNo','InvoiceDate'], axis=0, ascending=True, inplace=False,ignore_index=True)

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Buying_time,UnitPrice,CustomerID,Country
0,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010/12/1,08:26,7.65,17850.0,United Kingdom
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1,08:26,2.55,17850.0,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,2010/12/1,08:26,3.39,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1,08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1,08:26,3.39,17850.0,United Kingdom


In [6]:
#delete nall
df.dropna(axis=0, inplace=True)

In [7]:
len(df)

406827

In [8]:
df['monetary']=df.apply(lambda x: x['UnitPrice']*x['Quantity'], axis=1)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Buying_time,UnitPrice,CustomerID,Country,monetary
0,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010/12/1,08:26,7.65,17850.0,United Kingdom,15.30
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1,08:26,2.55,17850.0,United Kingdom,15.30
2,536365,71053,WHITE METAL LANTERN,6,2010/12/1,08:26,3.39,17850.0,United Kingdom,20.34
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1,08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1,08:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...
541902,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011/12/9,09:57,0.83,14397.0,United Kingdom,-9.13
541903,C581499,M,Manual,-1,2011/12/9,10:28,224.69,15498.0,United Kingdom,-224.69
541904,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011/12/9,11:57,10.95,15311.0,United Kingdom,-54.75
541905,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011/12/9,11:58,1.25,17315.0,United Kingdom,-6.25


In [9]:
from dateutil import parser

assess_date= parser.parse('2011-12-9')
start_date= parser.parse('2010-12-1')

#計算每個仁在三個產品的消費數量與時間
def RFM_cal(df,start_date, assess_date):
    df['values'] = 1
    
    df2 = df.pivot_table(index=['CustomerID','InvoiceDate'],
                              columns='Description',aggfunc=sum, values='values',fill_value=0,margins=1).fillna(0).reset_index()
    
    df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])
    
    df2 = df2[ (df2['InvoiceDate'] <= assess_date) &  (df2['InvoiceDate'] >= start_date)]
    
    # frequency'指消費者在一定期間內購買該產品的次數'
    df2['frequency'] = 1
    frequency = df2.groupby("CustomerID", as_index = False)['frequency'].sum()
    del df2['frequency']
    
    # recency'指消費者至今再次購買與上次購買產品的時間差'
    df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])
    recent_recency = df2.groupby("CustomerID", as_index = False)['InvoiceDate'].max()
    recent_recency['recency'] =( assess_date - recent_recency['InvoiceDate'] ).astype(str)
    
    # 問題：如何將recent_recency['recency']的days去除，並轉換為 int? 
    recent_recency['recency'] = recent_recency['recency'].str.replace('days.*', '', regex = True).astype(int)
    
  

    return df2,recent_recency, frequency


df2, recent_recency, frequency= RFM_cal(df, start_date, assess_date)

#recency merge 
df2 = recent_recency.merge(df2, on = ['CustomerID','InvoiceDate'])

#frequency merge
df2 = df2.merge(frequency, on = ['CustomerID'])



In [10]:
df2

Description,CustomerID,InvoiceDate,recency,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,...,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY,All,frequency
0,12347.0,2011-12-07,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,7
1,12348.0,2011-09-25,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,4
2,12349.0,2011-11-21,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,73,1
3,12350.0,2011-02-02,310,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,17,1
4,12352.0,2011-11-03,36,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,18280.0,2011-03-07,277,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,1
4367,18281.0,2011-06-12,180,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7,1
4368,18282.0,2011-12-02,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3
4369,18283.0,2011-12-06,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50,14


In [11]:
df2.sort_values(['CustomerID','InvoiceDate'],ascending=True).head(5)

Description,CustomerID,InvoiceDate,recency,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,...,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY,All,frequency
0,12347.0,2011-12-07,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,7
1,12348.0,2011-09-25,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,4
2,12349.0,2011-11-21,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,73,1
3,12350.0,2011-02-02,310,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,17,1
4,12352.0,2011-11-03,36,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,7


In [12]:
monetary = df.groupby('CustomerID', as_index = False)['monetary'].sum()
df2 = df2.merge(monetary, on = ['CustomerID'])
df2

,CustomerID,InvoiceDate,recency,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,...,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY,All,frequency,monetary
0,12347.0,2011-12-07,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,7,4310.00
1,12348.0,2011-09-25,75,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,4,1797.24
2,12349.0,2011-11-21,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,73,1,1757.55
3,12350.0,2011-02-02,310,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17,1,334.40
4,12352.0,2011-11-03,36,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15,7,1545.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,18280.0,2011-03-07,277,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10,1,180.60
4367,18281.0,2011-06-12,180,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7,1,80.82
4368,18282.0,2011-12-02,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,3,176.60
4369,18283.0,2011-12-06,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,50,14,2094.88


In [20]:
df2.to_csv("RFM.csv")